In [ ]:
!python -m pip install --upgrade numpy==1.13.3
#!python -m pip install --upgrade prompt-toolkit==1.0.15
!pip -q install ./python

from unityagents import UnityEnvironment
import numpy as np

# please do not modify the line below
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

from collections import deque
from IPython.display import clear_output

!curl -O https://raw.githubusercontent.com/udacity/workspaces-student-support/master/jupyter/workspace_utils.py


In [ ]:
import importlib
import dqn_agent
importlib.reload(dqn_agent)
from dqn_agent import Agent

brain_name = env.brain_names[0]
brain = env.brains[brain_name]


agent = Agent(state_size=state_size, action_size=action_size, seed=0)
agent.use_prioritized_ER(True)

def dqn(n_episodes=1500, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    scores_window = deque(maxlen=100)
    scores = []
    eps = eps_start
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0] 
        score = 0
        
        for t in range(max_t):
            #action = np.random.randint(action_size)        
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]        
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]                   
            done = env_info.local_done[0]
            
            agent.step(state, action, reward, next_state, done)
            score += reward                                
            state = next_state                             
            if done:                                       
                break
        scores_window.append(score)
        scores.append(score)
        #lear_output()
        eps = max(eps_end, eps_decay*eps)
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage 100 Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            if np.mean(scores_window)>=13:
                pass
                #print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            #torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            #break
                      
    return scores

In [ ]:
import matplotlib.pyplot as plt
from workspace_utils import active_session

with active_session():
    scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
import torch
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
print(y)
w = y.mean()
w.backward()
print(x.grad)
z = y * y * 3
out = z.mean()

print(z, out)
out.backward()
print(x.grad)